In [9]:
def correctLabels(datasetName, modelName):
    df1 = readDatasetWithSentenceId(datasetName, "test")
    df2 = pd.read_csv(f'results/{datasetName}/{datasetName}_{modelName }_predictions.csv')
    print(df1.labels.unique())
    print(df2.labels.unique())
    df1.labels= df1.labels.str.replace("I-", "").str.replace("B-", "")
    df2.labels= df2.labels.str.replace("I-", "").str.replace("B-", "")
    custom_labels = list(set(list(df1.labels.unique()) + list(df2.labels.unique())))
    report = classification_report(df1.labels, df2.labels, labels=custom_labels, target_names=custom_labels, output_dict=True)
    data = {
        "modelName": [modelName ],
        "datasetName": [datasetName],
        "accuracy_global": [round(report['weighted avg']['precision']*100, 2)],
        "recall_global": [round(report['weighted avg']['recall']*100, 2)],
        "f1_score_global": [round(report['weighted avg']['f1-score'] *100, 2)]
    }
    for label in custom_labels:
        data[f"accuracy_{label}"] = [round(report[label]['precision']*100, 2)]
        data[f"recall_{label}"] = [round(report[label]['recall']*100, 2)]
        data[f"f1_score_{label}"] = [round(report[label]['f1-score']*100, 2)]

    df = pd.DataFrame(data)
    df.to_csv(f'results/{datasetName}/{datasetName}_{modelName }_eval_corrected.csv', sep=",", index=False)
    return df

In [11]:
from usefull import correctLabels

df = correctLabels("ncbi-disease", "Bio-bert-based")
print(df.columns)

['O' 'B-Disease' 'I-Disease']
['O' 'B-Disease' 'I-Disease']
Index(['modelName', 'datasetName', 'accuracy_global', 'recall_global',
       'f1_score_global', 'accuracy_Disease', 'recall_Disease',
       'f1_score_Disease', 'accuracy_O', 'recall_O', 'f1_score_O'],
      dtype='object')


In [12]:
def plot_metrics(df, model_name, dataset_name, metric, show = True):
    results_folder = "results/"
    dataset_folder = f'{results_folder}{dataset_name}/'
    figures_folder = f'{dataset_folder}figures/'
    
    # Create directories if they don't exist
    for folder in [results_folder, dataset_folder, figures_folder]:
        if not os.path.isdir(folder): os.makedirs(folder)
    
    # Extract column names starting  with '_accuracy'
    metric_columns = [col for col in df.columns if col.startswith(f'{metric}_')]

    # Set up Seaborn with a diverging color palette
    sns.set(style="whitegrid")
    colors = sns.color_palette("RdYlGn", len(metric_columns))

    # Plotting the bar chart with seaborn
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(x=metric_columns, y=df.iloc[0][metric_columns], hue=metric_columns, palette=colors)

    # Customize plot appearance
    plt.title(f'{model_name} {metric} Comparison for {dataset_name}', fontsize=16)
    plt.xlabel('Entity type', fontsize=14)
    plt.ylabel(metric, fontsize=14)
    plt.ylim(0, 110)  # Set y-axis limit to percentages (0-100)

    # Rotate x-axis labels for better readability
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')

    # Add legend
    plt.legend([col.replace(f'{metric}_', "") for col in metric_columns], title="Labels List", loc="upper left")

    # Add grid lines
    plt.grid(True, linestyle='--', alpha=0.7)

    # Increase tick label font size
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    # Save the plot with tight layout to prevent label overlaps
    plt.tight_layout()
    if show == True : plt.show()
    plt.savefig(f'{figures_folder}{model_name}_{metric}_plot_for_{dataset_name}.png')


In [20]:
import os
from usefull import plot_metrics
model_list=["Bio-bert-based", "bert-based"]
metricList = ["accuracy", "recall", "f1_score"]
      for dataset in os.listdir("datasets/"):
        print(dataset)
        for model in model_list:
                df = correctLabels(datasetName = dataset)
                for metric in metricList:
                    plot_metrics(df, model_name=model, dataset_name=dataset, metric=metric, show = False)

IndentationError: unexpected indent (1639673148.py, line 5)

In [25]:
import os
from usefull import plot_metrics
model_list = ["Bio-bert-based", "bert-based"]
metricList = ["accuracy", "recall", "f1_score"]
for dataset in os.listdir("datasets/"):
    print(dataset)
    for model in model_list:
        df = correctLabels(datasetName = dataset, modelName = model)
        for metric in metricList:
            plot_metrics(df, model_name = model, dataset_name = dataset, metric = metric, show = False)

bc5cdr
['I-Entity' 'O' 'B-Entity']
['I-Entity' 'O' 'B-Entity']


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

<Figure size 1000x600 with 0 Axes>